In [441]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from ast import literal_eval
from forex_python.converter import CurrencyRates
import cpi

In [434]:
c = CurrencyRates()

In [358]:
path = r"./data_files/Phillips/"

In [359]:
df = pd.read_csv(path + "test_run.csv")
df.columns = [
    "artist", "title", "year_made", "medium", "size", "estimates", "sale_price", "sale_details", "img_url", "lot_url"
]
df.head()

,artist,title,year_made,medium,size,estimates,sale_price,sale_details,img_url,lot_url
0,Anselm Reyle,Untitled,2005,Mixed media on canvas in Plexiglas frame.,92 x 78 3/8 x 15 3/4 in. (233.7 x 199.1 x 40 cm).,"['£25,000', '35,000']","£31,200","['14', 'Oct', '2006,', '7pm', 'London']",https://assets.phillips.com/image/upload/t_Web...,https://www.phillips.com/detail/anselm-reyle/U...
1,Paul Pfeiffer,Four Horsemen of the Apocalypse (11),2004,Fujiflex digital c-print.,This work is from an edition of six and is acc...,"['£15,000', '20,000']","£16,800","['14', 'Oct', '2006,', '7pm', 'London']",https://assets.phillips.com/image/upload/t_Web...,https://www.phillips.com/detail/paul-pfeiffer/...
2,Tom Friedman,Untitled,1999,Wood and paint with wire support.,Sculpture: 1/16 x 1/16 x 3/16 in. (.2 x .2 x ....,"['£30,000', '40,000']","£34,800","['14', 'Oct', '2006,', '7pm', 'London']",https://assets.phillips.com/image/upload/t_Web...,https://www.phillips.com/detail/tom-friedman/U...
3,Johannes Kahrs,Untitled,1999,Oil on canvas.,Signed and dated twice “J Kahrs 1999” on the r...,"['£30,000', '40,000']","£78,000","['14', 'Oct', '2006,', '7pm', 'London']",https://assets.phillips.com/image/upload/t_Web...,https://www.phillips.com/detail/johannes-kahrs...
4,Ronan and Erwan Bouroullec,"""Honda"" vase",2001,"Painted fiberglass, plastic, lead; internal fl...","Produced by Galerie Kreo, Paris. Number 3 from...","['£10,000', '15,000']","£18,000","['14', 'Oct', '2006,', '7pm', 'London']",https://assets.phillips.com/image/upload/t_Web...,https://www.phillips.com/detail/ronan-and-erwa...


# Split estimate into high and low estimate and clean values

In [360]:
df['low_est'] = df['estimates'].apply(lambda x: x.split(" ")[0])
df['high_est'] = df['estimates'].apply(lambda x: x.split(" ")[-1])

In [361]:
df['low_est'] = df['low_est'].apply(lambda x: x.replace("[", "").replace(",", "").replace("'", "").replace("£", "").replace("$","").replace("HK", ""))

In [362]:
df['high_est'] = df['high_est'].apply(lambda x: x.replace("]", "").replace(",", "").replace("'", ""))

In [363]:
df['low_est'].replace('cant', np.nan, inplace = True)
df['low_est'].replace('on]', np.nan, inplace = True)

In [364]:
df['high_est'].replace('estimate', np.nan, inplace = True)
df['high_est'].replace('[on', np.nan, inplace = True)

In [365]:
df['high_est'].astype(float)

0       35000.0
1       20000.0
2       40000.0
3       40000.0
4       15000.0
         ...   
4689     1500.0
4690     3500.0
4691     3500.0
4692     1500.0
4693     3500.0
Name: high_est, Length: 4694, dtype: float64

# clean sale price and fill in 'cant find' with 0

In [366]:
df['sale_price'].value_counts()

not sold or cant find sale price    1067
£12,500                               41
£3,750                                41
£5,000                                39
£10,000                               37
                                    ... 
£116,500                               1
$141,600                               1
$4,820,000                             1
£951,000                               1
$1,000                                 1
Name: sale_price, Length: 1094, dtype: int64

In [367]:
df['sale_price'].replace("not sold or cant find sale price", 0, inplace = True)

In [368]:
df['sale_price'] = df['sale_price'].apply(lambda x: str(x).replace("$", "").replace("£", "").replace(",","").replace("HK", ""))

In [369]:
df['sale_price'].astype(float)

0       31200.0
1       16800.0
2       34800.0
3       78000.0
4       18000.0
         ...   
4689     3120.0
4690     2400.0
4691     6000.0
4692    13200.0
4693    13200.0
Name: sale_price, Length: 4694, dtype: float64

# Clean Sale_Details --> pull out Location and Date

In [370]:
df['sale_details'].replace("cant find sale location", np.nan, inplace = True)

In [371]:
temp = literal_eval(df['sale_details'][0])
temp

['14', 'Oct', '2006,', '7pm', 'London']

In [372]:
def get_month_year(x, months):
    try:
        temp = literal_eval(x)
        for i in range(len(temp)):
            if temp[i] in months:
                x = temp[i:i+2]

        return ' '.join(map(str, x)).replace(",","")
    except:
        pass

In [373]:
months = [
    'Jan', 'January',
    'Feb', 'February',
    'Mar', 'March',
    'Apr', 'April',
    'May',
    'June',
    'July',
    'Aug', 'August',
    'September',
    'Oct', 'October',
    'Nov', 'November',
    'Dec', 'December'
]

In [374]:
x = df['sale_details'][3000]
x

"['17', 'Nov', '2006,', '10am', '&', '2pm', 'New', 'York']"

In [375]:
df['sale_date'] = df['sale_details'].apply(lambda x: get_month_year(x, months))
df['sale_date']

0           Oct 2006
1           Oct 2006
2           Oct 2006
3           Oct 2006
4           Oct 2006
            ...     
4689    October 2007
4690    October 2007
4691    October 2007
4692    October 2007
4693    October 2007
Name: sale_date, Length: 4694, dtype: object

In [380]:
def get_year(x):  
    try:
        temp = x.split(" ")[-1]
        if temp == "May":
            pass
        elif temp == "5":
            pass
        else:
            return temp
    except:
        pass

def get_month(x):
    try:
        month = x.split(" ")[0]
        if len(month) > 3:
            month = month[:3]
        return month
    except:
        pass

In [381]:
df['sale_date'].apply(lambda x: get_year(x)).value_counts()

2008    975
2019    789
2007    644
2006    518
2009    444
2011    322
2010    255
2012    252
2017    107
2016     86
2015     66
2013     38
2020     38
2018     31
Name: sale_date, dtype: int64

In [382]:
df['sale_month'] = df['sale_date'].apply(lambda x: get_month(x))
df['sale_year'] = df['sale_date'].apply(lambda x: get_year(x))

In [383]:
locations = [
    "London",
    "Hong Kong",
    "New York",
    "Online",
    "Geneva"
]

In [384]:
temp = literal_eval(df['sale_details'][3000])
temp

['17', 'Nov', '2006,', '10am', '&', '2pm', 'New', 'York']

In [385]:
x = " ".join(map(str, temp))
x

'17 Nov 2006, 10am & 2pm New York'

In [386]:
def get_location(x):
    try:
        temp = " ".join(map(str, literal_eval(x)))

        for location in locations:
            if location in temp:
                return location
    except:
        pass

In [387]:
df['sale_location'] = df['sale_details'].apply(lambda x: get_location(x))
df['sale_location']

0       London
1       London
2       London
3       London
4       London
         ...  
4689    London
4690    London
4691    London
4692    London
4693    London
Name: sale_location, Length: 4694, dtype: object

In [388]:
df['sale_location'].value_counts()

London       2743
New York     1480
Hong Kong     281
Online        130
Geneva         13
Name: sale_location, dtype: int64

In [389]:
df = df[df['sale_location'] != 'Geneva']
df['sale_location'].value_counts()

London       2743
New York     1480
Hong Kong     281
Online        130
Name: sale_location, dtype: int64

In [390]:
test.reset_index(drop=True, inplace=True)

In [391]:
currency = [
    '£',
    '$',
    'HK$',
]

In [392]:
df['sale_location'].value_counts()

London       2743
New York     1480
Hong Kong     281
Online        130
Name: sale_location, dtype: int64

In [393]:
'$' in test['estimates'][100]

False

In [394]:
def set_currency(x):
    if x == "London":
        return "GBP"
    elif x == "New York":
        return "USD"
    elif x == "Hong Kong":
        return "HKD"
    elif x == "Online":
        return "Online"
    else:
        return np.nan

In [395]:
df['currency'] = df['sale_location'].apply(lambda x: set_currency(x))
df['currency']

0       GBP
1       GBP
2       GBP
3       GBP
4       GBP
       ... 
4689    GBP
4690    GBP
4691    GBP
4692    GBP
4693    GBP
Name: currency, Length: 4681, dtype: object

In [396]:
df['currency'].value_counts()

GBP       2743
USD       1480
HKD        281
Online     130
Name: currency, dtype: int64

In [413]:
data = df[['artist', 'sale_price', 'low_est', 'high_est', 'sale_month', 'sale_year', 'sale_location', 'currency']]
data

,artist,sale_price,low_est,high_est,sale_month,sale_year,sale_location,currency
0,Anselm Reyle,31200,25000,35000,Oct,2006,London,GBP
1,Paul Pfeiffer,16800,15000,20000,Oct,2006,London,GBP
2,Tom Friedman,34800,30000,40000,Oct,2006,London,GBP
3,Johannes Kahrs,78000,30000,40000,Oct,2006,London,GBP
4,Ronan and Erwan Bouroullec,18000,10000,15000,Oct,2006,London,GBP
...,...,...,...,...,...,...,...,...
4689,Afrika (Sergei Bugaev),3120,1000,1500,Oct,2007,London,GBP
4690,Anatoly Shuravlev,2400,2500,3500,Oct,2007,London,GBP
4691,Anatoly Shuravlev,6000,2500,3500,Oct,2007,London,GBP
4692,Andrei Filippov,13200,1000,1500,Oct,2007,London,GBP


In [414]:
data.isnull().sum()

artist             0
sale_price         0
low_est           56
high_est          56
sale_month        47
sale_year        129
sale_location     47
currency          47
dtype: int64

In [415]:
data.dropna(subset=['sale_year'], inplace=True)

C:\Users\Andrew\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [416]:
data.dropna(subset=['low_est'], inplace=True)

C:\Users\Andrew\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [417]:
data.isnull().sum()

artist           0
sale_price       0
low_est          0
high_est         0
sale_month       0
sale_year        0
sale_location    0
currency         0
dtype: int64

In [418]:
data = data.reset_index(drop=True)
data.shape

(4544, 8)

In [419]:
data

,artist,sale_price,low_est,high_est,sale_month,sale_year,sale_location,currency
0,Anselm Reyle,31200,25000,35000,Oct,2006,London,GBP
1,Paul Pfeiffer,16800,15000,20000,Oct,2006,London,GBP
2,Tom Friedman,34800,30000,40000,Oct,2006,London,GBP
3,Johannes Kahrs,78000,30000,40000,Oct,2006,London,GBP
4,Ronan and Erwan Bouroullec,18000,10000,15000,Oct,2006,London,GBP
...,...,...,...,...,...,...,...,...
4539,Afrika (Sergei Bugaev),3120,1000,1500,Oct,2007,London,GBP
4540,Anatoly Shuravlev,2400,2500,3500,Oct,2007,London,GBP
4541,Anatoly Shuravlev,6000,2500,3500,Oct,2007,London,GBP
4542,Andrei Filippov,13200,1000,1500,Oct,2007,London,GBP


In [420]:
data['currency'].value_counts()

GBP       2741
USD       1443
HKD        230
Online     130
Name: currency, dtype: int64

In [421]:
data[data['sale_location'] == 'Online']

,artist,sale_price,low_est,high_est,sale_month,sale_year,sale_location,currency
1102,Christina Quarles,8125,6000,8000,Feb,2019,Online,Online
1103,Lily Stockman,6250,8000,12000,Feb,2019,Online,Online
1104,Dan Colen,2500,2000,3000,Feb,2019,Online,Online
1105,Peter Wegner,4375,6000,8000,Feb,2019,Online,Online
1106,Elad Lassry,4750,5000,7000,Feb,2019,Online,Online
...,...,...,...,...,...,...,...,...
1897,Anne Hardy,1000,1000,1500,Jul,2019,Online,Online
1898,David Kim Whittaker,1500,2000,3000,Jul,2019,Online,Online
1899,KAWS,2500,1000,2000,Jul,2019,Online,Online
1900,KAWS,2000,1000,1500,Jul,2019,Online,Online


In [422]:
online_data = data[data['sale_location'] == 'Online']
location_data = data[data['sale_location'] != 'Online']

In [423]:
online_df = df[df['sale_location'] == 'Online']

In [424]:
online_currency = []

df_estimates = online_df['estimates'].tolist()

for est in df_estimates:
    if '$' in est:
        online_currency.append('USD')
    elif '£' in est:
        online_currency.append('GBP')
    elif 'HK$' in est:
        online_currency.append('HKD')
    else:
        online_currency.append(np.nan)

In [425]:
online_df['currency'] = online_currency
online_df['currency'].value_counts()

C:\Users\Andrew\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


USD    68
GBP    62
Name: currency, dtype: int64

In [426]:
online_data['currency'] = online_currency
online_data

C:\Users\Andrew\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,artist,sale_price,low_est,high_est,sale_month,sale_year,sale_location,currency
1102,Christina Quarles,8125,6000,8000,Feb,2019,Online,USD
1103,Lily Stockman,6250,8000,12000,Feb,2019,Online,USD
1104,Dan Colen,2500,2000,3000,Feb,2019,Online,USD
1105,Peter Wegner,4375,6000,8000,Feb,2019,Online,USD
1106,Elad Lassry,4750,5000,7000,Feb,2019,Online,USD
...,...,...,...,...,...,...,...,...
1897,Anne Hardy,1000,1000,1500,Jul,2019,Online,GBP
1898,David Kim Whittaker,1500,2000,3000,Jul,2019,Online,GBP
1899,KAWS,2500,1000,2000,Jul,2019,Online,GBP
1900,KAWS,2000,1000,1500,Jul,2019,Online,GBP


In [454]:
clean_data = pd.concat([location_data, online_data])
clean_data.head()

,artist,sale_price,low_est,high_est,sale_month,sale_year,sale_location,currency
0,Anselm Reyle,31200,25000,35000,Oct,2006,London,GBP
1,Paul Pfeiffer,16800,15000,20000,Oct,2006,London,GBP
2,Tom Friedman,34800,30000,40000,Oct,2006,London,GBP
3,Johannes Kahrs,78000,30000,40000,Oct,2006,London,GBP
4,Ronan and Erwan Bouroullec,18000,10000,15000,Oct,2006,London,GBP


In [455]:
clean_data.shape

(4544, 8)

In [456]:
def month_string_to_number(string):
    m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
         'may':5,
         'jun':6,
         'jul':7,
         'aug':8,
         'sep':9,
         'oct':10,
         'nov':11,
         'dec':12
        }
    s = string.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month')

In [457]:
clean_data['sale_month_num'] = clean_data['sale_month'].apply(lambda x: month_string_to_number(x))
clean_data.head()

,artist,sale_price,low_est,high_est,sale_month,sale_year,sale_location,currency,sale_month_num
0,Anselm Reyle,31200,25000,35000,Oct,2006,London,GBP,10
1,Paul Pfeiffer,16800,15000,20000,Oct,2006,London,GBP,10
2,Tom Friedman,34800,30000,40000,Oct,2006,London,GBP,10
3,Johannes Kahrs,78000,30000,40000,Oct,2006,London,GBP,10
4,Ronan and Erwan Bouroullec,18000,10000,15000,Oct,2006,London,GBP,10


# Get Currency conversion rates and conver sale_price, low_est, and high_est to USD

In [461]:
sale = int(clean_data['sale_price'][0])
year = int(clean_data['sale_year'][0])
month = int(clean_data['sale_month_num'][0])
curr = clean_data['currency'][0]

sale, year, month, curr

(31200, 2006, 10, 'GBP')

In [462]:
t = datetime(year, month, 1)

In [463]:
c.get_rate(curr, 'USD', t)

1.8680832227

In [474]:
c.convert(curr, 'USD', int(sale), t)

58284.19654824

In [483]:
convert_sale = []
convert_low = []
convert_high = []

for i in range(len(clean_data)):
    # get row data
    temp = clean_data.loc[i].tolist()
    # set sale price, low est, high est, currency, and datetime object
    sale = int(temp[1])
    low_est = int(temp[2])
    high_est = int(temp[3])
    curr = temp[-2]
    t = datetime(int(temp[5]), temp[-1], 1)
    
    # append converted value to list
    convert_sale.append(c.convert(curr, 'USD', sale, t))
    convert_low.append(c.convert(curr, 'USD', low_est, t))
    convert_high.append(c.convert(curr, 'USD', high_est, t))
    
len(convert_sale), len(convert_low), len(convert_high)

(4544, 4544, 4544)

In [485]:
clean_data['converted_sale'] = convert_sale
clean_data['converted_low_est'] = convert_low
clean_data['converted_high_est'] = convert_high

clean_data.head()

,artist,sale_price,low_est,high_est,sale_month,sale_year,sale_location,currency,sale_month_num,converted_sale,converted_low_est,converted_high_est
0,Anselm Reyle,31200,25000,35000,Oct,2006,London,GBP,10,58284.196548,46702.080568,65382.912795
1,Paul Pfeiffer,16800,15000,20000,Oct,2006,London,GBP,10,31383.798141,28021.248340,37361.664454
2,Tom Friedman,34800,30000,40000,Oct,2006,London,GBP,10,65009.296150,56042.496681,74723.328908
3,Johannes Kahrs,78000,30000,40000,Oct,2006,London,GBP,10,145710.491371,56042.496681,74723.328908
4,Ronan and Erwan Bouroullec,18000,10000,15000,Oct,2006,London,GBP,10,33625.498009,18680.832227,28021.248340


In [487]:
clean_data[['sale_price', 'currency', 'converted_sale']][clean_data['sale_location']=='New York']

,sale_price,currency,converted_sale
264,137000,USD,137000.000000
265,68750,USD,68750.000000
266,137000,USD,137000.000000
267,0,USD,0.000000
268,665000,USD,665000.000000
...,...,...,...
4130,0,USD,0.000000
4131,206500,USD,261104.453438
4132,92500,USD,189894.147955
4133,182500,USD,113936.488773


In [492]:
clean_data['converted_sale'] = np.where(clean_data['currency'] == 'USD', clean_data['sale_price'], clean_data['converted_sale'])
clean_data['converted_low_est'] = np.where(clean_data['currency'] == 'USD', clean_data['low_est'], clean_data['converted_low_est'])
clean_data['converted_high_est'] = np.where(clean_data['currency'] == 'USD', clean_data['high_est'], clean_data['converted_high_est'])

In [494]:
clean_data[['sale_price', 'currency', 'converted_sale', 'sale_year']][clean_data['sale_location']=='New York']

,sale_price,currency,converted_sale,sale_year
264,137000,USD,137000,2015
265,68750,USD,68750,2015
266,137000,USD,137000,2015
267,0,USD,0,2015
268,665000,USD,665000,2015
...,...,...,...,...
4130,0,USD,0,2012
4131,206500,USD,206500,2012
4132,92500,USD,92500,2012
4133,182500,USD,182500,2012


# use converted sales, low and high estimate to calculate inflation adjusted prices

In [500]:
def inflation_adj(sale, year):
    try:
        return cpi.inflate(int(sale), int(year))
    except:
        return sale

In [503]:
clean_data['sale_price_adj'] = clean_data.apply(lambda x: inflation_adj(x['converted_sale'], x['sale_year']), axis=1)
clean_data['low_est_adj'] = clean_data.apply(lambda x: inflation_adj(x['converted_low_est'], x['sale_year']), axis=1)
clean_data['high_est_adj'] = clean_data.apply(lambda x: inflation_adj(x['converted_high_est'], x['sale_year']), axis=1)

In [504]:
clean_data.head()

,artist,sale_price,low_est,high_est,sale_month,sale_year,sale_location,currency,sale_month_num,converted_sale,converted_low_est,converted_high_est,sale_price_adj,low_est_adj,high_est_adj
0,Anselm Reyle,31200,25000,35000,Oct,2006,London,GBP,10,58284.2,46702.1,65382.9,73912.3,59224.7,82913.5
1,Paul Pfeiffer,16800,15000,20000,Oct,2006,London,GBP,10,31383.8,28021.2,37361.7,39798,35534.5,47379
2,Tom Friedman,34800,30000,40000,Oct,2006,London,GBP,10,65009.3,56042.5,74723.3,82440.5,71069.1,94759.2
3,Johannes Kahrs,78000,30000,40000,Oct,2006,London,GBP,10,145710,56042.5,74723.3,184781,71069.1,94759.2
4,Ronan and Erwan Bouroullec,18000,10000,15000,Oct,2006,London,GBP,10,33625.5,18680.8,28021.2,42641.2,23688.9,35534.5


# Add a performance metric  => sale_price / low_est

In [508]:
e = 0.00001
clean_data['performance'] = clean_data.apply(lambda x: float(x['sale_price_adj']) / (float(x['low_est_adj']) + e), axis=1)

In [510]:
clean_data.head()

,artist,sale_price,low_est,high_est,sale_month,sale_year,sale_location,currency,sale_month_num,converted_sale,converted_low_est,converted_high_est,sale_price_adj,low_est_adj,high_est_adj,performance
0,Anselm Reyle,31200,25000,35000,Oct,2006,London,GBP,10,58284.2,46702.1,65382.9,73912.3,59224.7,82913.5,1.247998
1,Paul Pfeiffer,16800,15000,20000,Oct,2006,London,GBP,10,31383.8,28021.2,37361.7,39798,35534.5,47379,1.119981
2,Tom Friedman,34800,30000,40000,Oct,2006,London,GBP,10,65009.3,56042.5,74723.3,82440.5,71069.1,94759.2,1.160005
3,Johannes Kahrs,78000,30000,40000,Oct,2006,London,GBP,10,145710,56042.5,74723.3,184781,71069.1,94759.2,2.600014
4,Ronan and Erwan Bouroullec,18000,10000,15000,Oct,2006,London,GBP,10,33625.5,18680.8,28021.2,42641.2,23688.9,35534.5,1.800054


In [512]:
clean_data['performance'].describe()

count    4544.000000
mean        1.408725
std         1.325707
min         0.000000
25%         0.818625
50%         1.249897
75%         1.785710
max        14.133333
Name: performance, dtype: float64

# save the cleanded data

In [513]:
final_data = clean_data[[
    'artist',
    'sale_year',
    'sale_location',
    'sale_price_adj',
    'low_est_adj',
    'high_est_adj',
    'performance'
]]

final_data.shape

(4544, 7)

In [514]:
final_data.head()

,artist,sale_year,sale_location,sale_price_adj,low_est_adj,high_est_adj,performance
0,Anselm Reyle,2006,London,73912.3,59224.7,82913.5,1.247998
1,Paul Pfeiffer,2006,London,39798,35534.5,47379,1.119981
2,Tom Friedman,2006,London,82440.5,71069.1,94759.2,1.160005
3,Johannes Kahrs,2006,London,184781,71069.1,94759.2,2.600014
4,Ronan and Erwan Bouroullec,2006,London,42641.2,23688.9,35534.5,1.800054


In [522]:
final_data.to_csv(path+'clean_adj_sale_prices.csv', index=False)

In [531]:
final_data['sale_price_adj'] = pd.to_numeric(final_data['sale_price_adj'])
final_data['sale_year'] = pd.to_numeric(final_data['sale_year'])
final_data['low_est_adj'] = pd.to_numeric(final_data['low_est_adj'])
final_data['high_est_adj'] = pd.to_numeric(final_data['high_est_adj'])
final_data['performance'] = pd.to_numeric(final_data['performance'])

C:\Users\Andrew\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Andrew\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Andrew\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [535]:
final_data.groupby(['sale_location'])['sale_price_adj'].sum().map('${:,.2f}'.format)

sale_location
Hong Kong     $45,262,361.32
London       $916,244,809.33
New York     $718,106,673.69
Online         $7,454,780.00
Name: sale_price_adj, dtype: object